# Install, Paths and Parameters

In [31]:
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm
import random
import sys

from torch.utils.tensorboard import SummaryWriter

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
# from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import *
from src.model.train import *
from src.model.multihead_model import *

from torchattacks import *
from sklearn import preprocessing

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')

LOG_BASE_PATH = Path(MAX_PATH, 'logs')

# DamageNet
DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')

# Image Net
ORI_PATH = Path(DATA_PATH, 'ori_data')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

VAL_PATH = Path(ORI_PATH, 'validation')
VAL_IMAGES_PATH = Path(VAL_PATH,'images')
VAL_LABEL_PATH = Path(VAL_PATH, 'correct_labels.txt')

TRAIN_PATH = Path(ORI_PATH, 'train')
TRAIN_IMAGES_PATH = Path(TRAIN_PATH,'images')
TRAIN_LABEL_PATH = Path(TRAIN_PATH, 'correct_labels.txt')

# Adversarial Data
# PGD
PGD_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'pgd_06', 'train')
PGD_TRAIN_IMAGES_PATH = Path(PGD_TRAIN_PATH,'images')
PGD_TRAIN_LABEL_PATH = Path(PGD_TRAIN_PATH, 'labels.txt')

PGD_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'pgd_06', 'validation')
PGD_VAL_IMAGES_PATH = Path(PGD_VAL_PATH,'images')
PGD_VAL_LABEL_PATH = Path(PGD_VAL_PATH, 'labels.txt')

# CW
CW_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'cw', 'train')
CW_TRAIN_IMAGES_PATH = Path(CW_TRAIN_PATH,'images')
CW_TRAIN_LABEL_PATH = Path(CW_TRAIN_PATH, 'labels.txt')

CW_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'cw', 'validation')
CW_VAL_IMAGES_PATH = Path(CW_VAL_PATH,'images')
CW_VAL_LABEL_PATH = Path(CW_VAL_PATH, 'labels.txt')

# FGSM
FGSM_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'fgsm_06', 'train')
FGSM_TRAIN_IMAGES_PATH = Path(FGSM_TRAIN_PATH,'images')
FGSM_TRAIN_LABEL_PATH = Path(FGSM_TRAIN_PATH, 'labels.txt')

FGSM_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'fgsm_06', 'validation')
FGSM_VAL_IMAGES_PATH = Path(FGSM_VAL_PATH,'images')
FGSM_VAL_LABEL_PATH = Path(FGSM_VAL_PATH, 'labels.txt')


# TB LOG
TB_LOGS_BASE_PATH = Path(LOG_BASE_PATH, 'tb_logs')


# Model save path
ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH = Path(MAX_PATH, 'adversarial_data', 'adv_classifiers')

In [32]:
ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH

PosixPath('/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers')

In [33]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
# INDEX_SUBSET = get_random_indexes(number_of_images = 50000, n_samples=1000)
# CLASS_SUBSET = get_random_classes(number_of_classes = 25, min_rand_class = 1, max_rand_class = 1001)


CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)

# CLASS_SUBSET = CLASS_SUBSET[:25] 

NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 16

DEVICE = 'cuda'

In [34]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [35]:
model, linear_classifier = get_dino()

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


# Load data

In [36]:
# Remember to set the correct transformation
# encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit([i for i in CLASS_SUBSET])


# PGD
pgd_train_dataset = AdvTrainingImageDataset(PGD_TRAIN_IMAGES_PATH, 
                                            PGD_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                            label_encoder=label_encoder
                                           )

pgd_val_dataset = AdvTrainingImageDataset(PGD_VAL_IMAGES_PATH, 
                                          PGD_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                          label_encoder=label_encoder
                                         )


pgd_train_loader = DataLoader(pgd_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

pgd_val_loader = DataLoader(pgd_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# CW
cw_train_dataset = AdvTrainingImageDataset(CW_TRAIN_IMAGES_PATH, 
                                            CW_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                           label_encoder=label_encoder
                                          )

cw_val_dataset = AdvTrainingImageDataset(CW_VAL_IMAGES_PATH, 
                                          CW_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                         label_encoder=label_encoder
                                        )

cw_train_loader = DataLoader(cw_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

cw_val_loader = DataLoader(cw_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# FGSM
fgsm_train_dataset = AdvTrainingImageDataset(FGSM_TRAIN_IMAGES_PATH, 
                                            FGSM_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                            label_encoder=label_encoder
                                            )

fgsm_val_dataset = AdvTrainingImageDataset(FGSM_VAL_IMAGES_PATH, 
                                          FGSM_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                          label_encoder=label_encoder
                                          )

fgsm_train_loader = DataLoader(fgsm_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

fgsm_val_loader = DataLoader(fgsm_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# Clean
clean_train_dataset = ImageDataset(TRAIN_IMAGES_PATH, 
                                  TRAIN_LABEL_PATH, 
                                  ORIGINAL_TRANSFORM,
                                  CLASS_SUBSET, 
                                  index_subset=None, 
                                  label_encoder=label_encoder)

clean_val_dataset = ImageDataset(VAL_IMAGES_PATH, 
                                  VAL_LABEL_PATH, 
                                  ORIGINAL_TRANSFORM,
                                  CLASS_SUBSET, 
                                  index_subset=None, 
                                  label_encoder=label_encoder)

clean_train_loader = DataLoader(clean_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

clean_val_loader = DataLoader(clean_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY,
                            shuffle=False)


loader_dict = {
    "pgd" : {
        "train" : pgd_train_loader,
        "validation" : pgd_val_loader,
    },
    "cw" : {
        "train" : cw_train_loader,
        "validation" : cw_val_loader,
    }, 
    "fgsm" : {
        "train" : fgsm_train_loader,
        "validation" : fgsm_val_loader,
    },
    "clean" : {
        "train" : clean_train_loader,
        "validation" : clean_val_loader,
    },
}




In [37]:
loader_dict["pgd"]["train"]

In [38]:
version = '25_classes'

## Classifier

In [39]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)


In [40]:
# Linear Binary Classifier
class LinearBC(nn.Module):
    def __init__(self, input_shape):
        super(LinearBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

## Train various classifiers on all adversarial datasets

In [42]:
for attack, loaders in loader_dict.items():
    if not attack == "clean":
        continue
        
    # Tensorboard summary writer
    TB_LOGS_PATH = Path(TB_LOGS_BASE_PATH, 'adversarial_classifier', version, attack)
    writer = SummaryWriter(TB_LOGS_PATH)
    np.set_printoptions(precision=4)
    
    # Initialise classifier
    adv_linear_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                         num_labels=len(CLASS_SUBSET))
    adv_linear_classifier = adv_linear_classifier.cuda()

    # Metric logger path
    LOG_PATH = Path(LOG_BASE_PATH, 'adv_classifier', version, attack)
    if not os.path.isdir(LOG_PATH):
        os.makedirs(LOG_PATH)
    
    # train
    print(f"Training classifier for {attack}")
    loggers = train(model, 
                    adv_linear_classifier, 
                    loaders["train"], 
                    loaders["validation"], 
                    log_dir=LOG_PATH, 
                    tensor_dir=None, 
                    optimizer=None, 
                    adversarial_attack=None, 
                    epochs=5, 
                    val_freq=1, 
                    batch_size=16,  
                    lr=0.001, 
                    to_restore = {"epoch": 0, "best_acc": 0.}, 
                    n=4, 
                    avgpool_patchtokens=False, 
                    show_image=False, 
                    writer=writer
                   )
    
    writer.close()
    
    # Save adversarial Classifier
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_file = f"{attack}.pt"
    torch.save(adv_linear_classifier.state_dict(), str(save_path) + "/" + save_file)
    print(f'Finished Training, saving to {str(save_path.stem)}/{save_file}')

Training classifier for clean
Epoch: [0]  [   0/2012]  eta: 0:07:42  lr: 0.000063  loss: 4.070327 (4.070327)  time: 0.229926  data: 0.190067  max mem: 210
Epoch: [0]  [  20/2012]  eta: 0:07:45  lr: 0.000063  loss: 3.364912 (3.396890)  time: 0.233918  data: 0.195904  max mem: 211
Epoch: [0]  [  40/2012]  eta: 0:07:25  lr: 0.000063  loss: 1.061000 (2.291339)  time: 0.217225  data: 0.179242  max mem: 211
Epoch: [0]  [  60/2012]  eta: 0:07:07  lr: 0.000063  loss: 0.637545 (1.764758)  time: 0.206038  data: 0.168053  max mem: 211
Epoch: [0]  [  80/2012]  eta: 0:06:56  lr: 0.000063  loss: 0.350218 (1.438930)  time: 0.204567  data: 0.166585  max mem: 211
Epoch: [0]  [ 100/2012]  eta: 0:06:53  lr: 0.000063  loss: 0.228549 (1.213146)  time: 0.219306  data: 0.181325  max mem: 211
Epoch: [0]  [ 120/2012]  eta: 0:06:44  lr: 0.000063  loss: 0.336601 (1.068877)  time: 0.201224  data: 0.163227  max mem: 211
Epoch: [0]  [ 140/2012]  eta: 0:06:38  lr: 0.000063  loss: 0.299253 (0.958927)  time: 0.205310 

## Evaluation

### Evaluate on all adversarial datasets

In [22]:
attacks = [x for x in loader_dict.keys()]

for attack in attacks:
    
    print("#"*50 + f" evaluating adv_classifier trained on {attack} " + "#"*50)
    adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                             num_labels=len(CLASS_SUBSET))
    adv_classifier.to(device)
    
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    save_file = f"{attack}.pt"
    adv_classifier.load_state_dict(torch.load(str(save_path) + "/" + save_file))
    
    for applied_attack in attacks:
        
#         if applied_attack == attack:
#             continue
        
        print("-"*50 + f" {applied_attack} dataset " + "-"*50)
        logger_dict, logger = validate_network(model, 
                                               adv_classifier, 
                                               loader_dict[applied_attack]["validation"], 
                                               tensor_dir=None, 
                                               adversarial_attack=None, 
                                               n=4, 
                                               avgpool=False)


################################################## evaluating adv_classifier trained on pgd ##################################################
-------------------------------------------------- pgd dataset --------------------------------------------------
Test:  [ 0/32]  eta: 0:00:02  loss: 0.071055 (0.071055)  acc1: 100.000000 (100.000000)  acc5: 100.000000 (100.000000)  time: 0.084277  data: 0.046243  max mem: 210
Test:  [20/32]  eta: 0:00:00  loss: 0.116650 (0.146619)  acc1: 93.750000 (95.833333)  acc5: 100.000000 (100.000000)  time: 0.077803  data: 0.045770  max mem: 210
Test:  [31/32]  eta: 0:00:00  loss: 0.025480 (0.119763)  acc1: 100.000000 (96.200000)  acc5: 100.000000 (100.000000)  time: 0.074005  data: 0.044119  max mem: 210
Test: Total time: 0:00:02 (0.076153 s / it)
* Acc@1 96.200 Acc@5 100.000 loss 0.120
-------------------------------------------------- cw dataset --------------------------------------------------
Test:  [ 0/32]  eta: 0:00:02  loss: 0.103200 (0.103200)  

### Evaluate on newly generated attacks

In [ ]:
attacks = [x for x in loader_dict.keys()]
for attack in attacks:
    print("#"*50 + f" evaluating adv_classifier trained on {attack} " + "#"*50)
    adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                             num_labels=len(CLASS_SUBSET))
    adv_classifier.to(device)
    
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    save_file = f"{attack}.pt"
    adv_classifier.load_state_dict(torch.load(str(save_path) + "/" + save_file))
    
    vits = ViTWrapper(model, adv_classifier, transform=None)

    for applied_attack in attacks:
        
        if applied_attack == "pgd":
            ev_attack = PGD(vits, eps=0.3, alpha=6/255, steps=15)
        elif applied_attack == "cw":
            ev_attack = CW(vits, c=10, lr=0.003, steps=30)
        elif applied_attack == "fgsm":
            ev_attack = FGSM(vits, eps=0.03)
        else:
            continue
        
        print("-"*50 + f" applying attack: {ev_attack} " + "-"*50)
        logger_dict, logger = validate_network(model, 
                                               adv_classifier, 
                                               loader_dict["clean"]["validation"], 
                                               tensor_dir=None, 
                                               adversarial_attack=ev_attack,
                                               n=4, 
                                               avgpool=False)

### Evaluate on full pipeline with post-hoc as multiplexer

In [11]:
# Load posthoc
name="cw" # cw, fgsm, pgd
posthoc = LinearBC(1536)
posthoc.to(device)
posthoc.load_state_dict(torch.load(f"/cluster/scratch/mmathys/dl_data/posthoc-models/{name}.pt"))

<All keys matched successfully>

In [12]:
# Load best adv_classifier
name="cw" # cw, fgsm, pgd
adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                  num_labels=len(CLASS_SUBSET))
adv_classifier.to(device)
adv_classifier.load_state_dict(torch.load(f"/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/{version}/{name}.pt"))

<All keys matched successfully>

In [13]:
# Load clean_classifier
name="clean"
clean_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                    num_labels=len(CLASS_SUBSET))
clean_classifier.to(device)
clean_classifier.load_state_dict(torch.load(f"/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/{version}/{name}.pt"))

<All keys matched successfully>

In [17]:
# Perform validation on clean dataset
for attack, loaders in loader_dict.items():
    print("#"*50 + f" Validating on {attack} " + "#"*50)
    log_dict, logger = validate_multihead_network(model, 
                                                  posthoc,
                                                  adv_classifier,
                                                  clean_classifier,
                                                  loader_dict[attack]["validation"], 
                                                  tensor_dir=None, 
                                                  adversarial_attack=None, 
                                                  n=4, 
                                                  avgpool=False)

################################################## Validating on pgd ##################################################
Test:  [ 0/32]  eta: 0:00:03  loss: 0.293546 (0.293546)  acc1: 93.750000 (93.750000)  acc5: 100.000000 (100.000000)  time: 0.094445  data: 0.045961  max mem: 210
Test:  [ 5/32]  eta: 0:00:02  loss: 0.293546 (0.300796)  acc1: 93.750000 (92.708333)  acc5: 100.000000 (98.958333)  time: 0.089142  data: 0.047695  max mem: 210
Test:  [10/32]  eta: 0:00:01  loss: 0.340758 (0.394105)  acc1: 87.500000 (89.204545)  acc5: 100.000000 (98.295455)  time: 0.084818  data: 0.046690  max mem: 210
Test:  [15/32]  eta: 0:00:01  loss: 0.340758 (0.366558)  acc1: 87.500000 (89.453125)  acc5: 100.000000 (98.828125)  time: 0.082831  data: 0.046439  max mem: 210
Test:  [20/32]  eta: 0:00:00  loss: 0.340758 (0.350821)  acc1: 87.500000 (89.880952)  acc5: 100.000000 (99.107143)  time: 0.081223  data: 0.046395  max mem: 210
Test:  [25/32]  eta: 0:00:00  loss: 0.350162 (0.347260)  acc1: 87.500000 (

## Black Box on Multihead model